In [8]:
import argparse
import pathlib
import random
import sys
import warnings

import numpy as np
import pandas as pd
import pycytominer.aggregate

# Suppress all RuntimeWarnings
warnings.filterwarnings("ignore", category=RuntimeWarning)
import itertools

sys.path.append("../utils")
from mAP_utils import run_mAP_across_time

# check if in a jupyter notebook
try:
    cfg = get_ipython().config
    in_notebook = True
except NameError:
    in_notebook = False

In [9]:
data_file_path = pathlib.Path(
    "../../data/CP_aggregated/profiles/aggregated_profile.parquet"
).resolve(strict=True)
# load the dose information
well_dose_df = pd.read_parquet(
    pathlib.Path(
        "../../data/CP_feature_select/profiles/features_selected_profile.parquet"
    ).resolve(strict=True),
    columns=["Metadata_Well", "Metadata_dose"],
).drop_duplicates()


pathlib.Path("../results").mkdir(exist_ok=True)
# output filepath
output_file_path = pathlib.Path("../results/mAP_across_channels.parquet").resolve()
aggregate_df = pd.read_parquet(data_file_path)
print(aggregate_df.shape)
# drop rows with nan in features (non Metadata)
aggregate_df = aggregate_df.dropna(
    subset=aggregate_df.filter(regex="^(?!Metadata)").columns
)
# merge the dose information with the profile data
aggregate_df = aggregate_df.merge(
    well_dose_df,
    left_on="Metadata_Well",
    right_on="Metadata_Well",
    how="left",
)
dose = aggregate_df.pop("Metadata_dose")
aggregate_df.insert(
    1,
    "Metadata_dose",
    dose,
)
print(aggregate_df.shape)
aggregate_df.head()

(390, 813)
(390, 814)


,Metadata_Well,Metadata_dose,Metadata_Time,Cytoplasm_AreaShape_Area,Cytoplasm_AreaShape_Compactness,Cytoplasm_AreaShape_Eccentricity,Cytoplasm_AreaShape_Extent,Cytoplasm_AreaShape_FormFactor,Cytoplasm_AreaShape_MeanRadius,Cytoplasm_AreaShape_Orientation,...,Nuclei_Texture_DifferenceVariance_CL_488_1_3_00_256,Nuclei_Texture_DifferenceVariance_CL_488_2_3_01_256,Nuclei_Texture_DifferenceVariance_CL_561_3_02_256,Nuclei_Texture_InverseDifferenceMoment_CL_488_1_3_01_256,Nuclei_Texture_InverseDifferenceMoment_CL_561_3_01_256,Nuclei_Texture_InverseDifferenceMoment_DNA_3_00_256,Nuclei_Texture_SumAverage_CL_488_1_3_03_256,Nuclei_Texture_SumAverage_CL_488_2_3_03_256,Nuclei_Texture_SumAverage_CL_561_3_02_256,Nuclei_Texture_SumAverage_DNA_3_00_256
0,C-02,0.0,0.0,-0.258751,0.156617,0.293786,-0.010669,-0.324981,-0.209535,0.042542,...,0.034509,0.516061,0.030412,0.365083,0.422430,0.297776,-0.155818,0.381277,0.052962,0.158581
1,C-02,0.0,1.0,0.173437,-0.034522,0.261068,0.109414,-0.165482,0.141825,0.126173,...,0.412496,0.350681,-0.475927,0.436283,0.198046,0.297776,-0.189419,0.381277,0.267969,0.158581
2,C-02,0.0,2.0,0.253716,0.021714,0.332154,0.020295,-0.213724,0.169555,0.070330,...,0.451687,-0.328098,-0.500268,0.448701,0.187447,0.297776,-0.212391,0.381277,0.263123,0.158581
3,C-02,0.0,3.0,0.277141,0.172825,0.470829,-0.069799,-0.337952,0.120320,0.037571,...,0.408877,-0.890800,-0.614235,0.444435,0.134378,0.297776,-0.210890,0.381277,0.289399,0.158581
4,C-02,0.0,4.0,0.278720,0.163328,0.523522,-0.077466,-0.330367,0.206325,0.030241,...,0.277523,-1.464788,-0.655018,0.395577,0.138285,0.297776,-0.205587,-1.424332,0.334251,0.158581


In [10]:
# get the non metadata columns
metadata_cols = [cols for cols in aggregate_df.columns if "Metadata" in cols]
features_cols = [cols for cols in aggregate_df.columns if "Metadata" not in cols]
# drop features that contain AreaShape in the feature name
non_feature_cols = [cols for cols in features_cols if "AreaShape" in cols]
non_feature_cols = (
    non_feature_cols
    + [col for col in features_cols if "Location" in col]
    + [col for col in features_cols if "Neighbors" in col]
)
correlation_cols = [col for col in features_cols if "Correlation" in col]
features_cols = [cols for cols in features_cols if cols not in non_feature_cols]
features_cols = [cols for cols in features_cols if cols not in correlation_cols]
df = pd.DataFrame(features_cols, columns=["features"])

df[
    [
        "Compartment",
        "Feature_type",
        "Measurement",
        "Channel",
        "misc1",
        "misc2",
        "misc3",
        "misc4",
        "misc5",
    ]
] = df["features"].str.split("_", expand=True)
# Update Channel column where it contains "CL"

df.loc[df["Channel"].str.contains("CL", na=False), "Channel"] = (
    df["Channel"] + "_" + df["misc1"]
)
# Update Channel column where it contains "CL_488"
df.loc[df["Channel"].str.contains("CL_488", na=False), "Channel"] = (
    df["Channel"] + "_" + df["misc2"]
)
df.drop(columns=["misc1", "misc2", "misc3", "misc4", "misc5"], inplace=True)
df.head()

,features,Compartment,Feature_type,Measurement,Channel
0,Cytoplasm_Granularity_1_CL_488_1,Cytoplasm,Granularity,1,CL_488_1
1,Cytoplasm_Granularity_1_CL_488_2,Cytoplasm,Granularity,1,CL_488_2
2,Cytoplasm_Granularity_1_CL_561,Cytoplasm,Granularity,1,CL_561
3,Cytoplasm_Granularity_1_DNA,Cytoplasm,Granularity,1,DNA
4,Cytoplasm_Intensity_IntegratedIntensityEdge_CL...,Cytoplasm,Intensity,IntegratedIntensityEdge,CL_488_2


In [11]:
# create a dictionary of loadable features for each channel
loadable_features = {}
for channel in df.Channel.unique():
    channel_df = df.query("Channel == @channel")
    loadable_features[channel] = channel_df.features.tolist()
loadable_features["None"] = non_feature_cols
loadable_features["All"] = [
    cols for cols in aggregate_df.columns if "Metadata" not in cols
]

In [12]:
unique_channels = df["Channel"].unique().tolist()
# unique_channels = unique_channels + ['None']
# channel combinations
channel_combinations = []
for i in range(1, len(unique_channels) + 1):
    channel_combinations.extend(list(itertools.combinations(unique_channels, i)))
channel_combinations = [list(comb) for comb in channel_combinations]
channel_combinations = channel_combinations + [["None"]]
channel_combinations

[['CL_488_1'],
 ['CL_488_2'],
 ['CL_561'],
 ['DNA'],
 ['CL_488_1', 'CL_488_2'],
 ['CL_488_1', 'CL_561'],
 ['CL_488_1', 'DNA'],
 ['CL_488_2', 'CL_561'],
 ['CL_488_2', 'DNA'],
 ['CL_561', 'DNA'],
 ['CL_488_1', 'CL_488_2', 'CL_561'],
 ['CL_488_1', 'CL_488_2', 'DNA'],
 ['CL_488_1', 'CL_561', 'DNA'],
 ['CL_488_2', 'CL_561', 'DNA'],
 ['CL_488_1', 'CL_488_2', 'CL_561', 'DNA'],
 ['None']]

In [ ]:
channel_combo_output_dict = {}
# loop through the channel combinations and shuffle
for shuffle in [False, True]:
    for channel_combination in channel_combinations:
        features_to_load = []
        features_to_load.append(metadata_cols)
        # rename the all channels to all
        if len(channel_combination) == 5:
            channel_combination = ["All"]
        elif channel_combination == ["None"]:
            features_to_load.append(loadable_features["None"])
        else:
            for channel in channel_combination:
                features_to_load.append(loadable_features[channel])
            features_to_load.append(loadable_features["None"])
        # flatten the list
        features_to_load = list(itertools.chain(*features_to_load))
        temporary_df = aggregate_df[features_to_load]
        # shuffle the data
        if shuffle == True:
            print("shuffled")
            shuffle_status = "shuffled"
            random.seed(0)
            # permutate the data
            for col in temporary_df.columns:
                if "Metadata_Time" in col or "Metadata_dose" in col:
                    continue
                else:
                    temporary_df[col] = np.random.permutation(temporary_df[col])
        else:
            print("not shuffled")
            shuffle_status = "non_shuffled"

        # run mAP with the 0 dose as the reference
        dict_of_map_dfs = run_mAP_across_time(
            temporary_df,
            seed=0,
            time_column="Metadata_Time",
            reference_column_name="Metadata_dose",
            reference_group=aggregate_df["Metadata_dose"].min(),
        )
        # concat and rename the columns
        df = pd.concat(dict_of_map_dfs.values(), keys=dict_of_map_dfs.keys())
        df.reset_index(inplace=True)
        df.rename(
            columns={
                "level_0": "Metadata_Time",
            },
            inplace=True,
        )
        df["Channel"] = "_".join(channel_combination)
        df["shuffle"] = shuffle_status
        channel_combo_output_dict[
            f"{'_'.join(channel_combination)}_{shuffle_status}"
        ] = df

not shuffled


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

...

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

In [ ]:
final_df = pd.concat(
    channel_combo_output_dict.values(), keys=channel_combo_output_dict.keys()
)
final_df.reset_index(inplace=True, drop=True)
# save the output to a file
final_df.to_parquet("../results/mAP_across_channels.parquet")
final_df.head()

,Metadata_Time,Metadata_dose,Metadata_reference_index,mean_average_precision,indices,p_value,corrected_p_value,below_p,below_corrected_p,-log10(p-value),Channel,shuffle
0,0.0,0.61,-1,1.0,"[0, 9, 18]",9.999990e-07,9.999990e-07,True,True,6.0,CL_488_1,non_shuffled
1,0.0,1.22,-1,1.0,"[1, 10, 19]",9.999990e-07,9.999990e-07,True,True,6.0,CL_488_1,non_shuffled
2,0.0,2.44,-1,1.0,"[2, 11, 20]",9.999990e-07,9.999990e-07,True,True,6.0,CL_488_1,non_shuffled
3,0.0,4.88,-1,1.0,"[3, 12, 21]",9.999990e-07,9.999990e-07,True,True,6.0,CL_488_1,non_shuffled
4,0.0,9.77,-1,1.0,"[4, 13, 22]",9.999990e-07,9.999990e-07,True,True,6.0,CL_488_1,non_shuffled
